In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('stud.csv')

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
x = df.drop('math_score', axis=1)
y = df['math_score']

In [5]:
num_features = x.select_dtypes(exclude='object').columns
cat_features = x.select_dtypes(include='object').columns

In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numerical_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', oh_transformer, cat_features),
        ('StandardScaler', numerical_transformer, num_features)
    ]
)

In [7]:
X = preprocessor.fit_transform(x)

In [8]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [9]:
x.shape

(1000, 7)

In [10]:
X.shape

(1000, 19)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x_train.shape, x_test.shape

((800, 19), (200, 19))

In [20]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, mse, r2_square

In [22]:
models = {
    'Linear Regression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'k_neigh':KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest':RandomForestRegressor(),
    'XGB': XGBRegressor(),
    'CatBoost':CatBoostRegressor(verbose=False),
    'Adaboost': AdaBoostRegressor()
}

model_list = []
r2_list = []
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train, y_train) # Train model
    
    #Make Prediction
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    #Evaluate Train and Test dataset
    model_train_mae, model_train_rmse, model_train_mse, model_train_r2 = evaluate_model(y_train, y_train_pred) 
    model_test_mae, model_test_rmse, model_test_mse, model_test_r2 = evaluate_model(y_test, y_test_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model perfomance for training set')
    print(" - Root Mean Squred error : {:.4f}".format(model_train_rmse))
    print('- mean absolute error {:.4f}'.format(model_test_mae))
    print(" - r2 score : {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print("="*25)
    print('\n')

Linear Regression
Model perfomance for training set
 - Root Mean Squred error : 5.3241
- mean absolute error 4.2170
 - r2 score : 0.8803


Lasso
Model perfomance for training set
 - Root Mean Squred error : 6.5938
- mean absolute error 5.1579
 - r2 score : 0.8253


Ridge
Model perfomance for training set
 - Root Mean Squred error : 5.3233
- mean absolute error 4.2111
 - r2 score : 0.8806


k_neigh
Model perfomance for training set
 - Root Mean Squred error : 5.6989
- mean absolute error 5.6360
 - r2 score : 0.7839


Decision Tree
Model perfomance for training set
 - Root Mean Squred error : 0.2795
- mean absolute error 6.2550
 - r2 score : 0.7486


Random Forest
Model perfomance for training set
 - Root Mean Squred error : 2.2995
- mean absolute error 4.6535
 - r2 score : 0.8525


XGB
Model perfomance for training set
 - Root Mean Squred error : 0.9087
- mean absolute error 5.0844
 - r2 score : 0.8216


CatBoost
Model perfomance for training set
 - Root Mean Squred error : 3.0427
- mea

In [25]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model_name', 'R2_Score']).sort_values(by=['R2_Score'], ascending=False).reset_index().drop('index', axis=1)

,Model_name,R2_Score
0,Ridge,0.880593
1,Linear Regression,0.880319
2,Random Forest,0.852470
3,CatBoost,0.851632
4,Adaboost,0.849363
5,Lasso,0.825320
6,XGB,0.821589
7,k_neigh,0.783876
8,Decision Tree,0.748642
